<a href="https://colab.research.google.com/github/ashishar/qbook/blob/master/Rydberg_reconstruction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Generative Modelling
# Tutorial 1: Rydberg wavefunction reconstruction using restricted Boltzmann machines

Perimeter Institute

Quantum and AI Career Trajectories Mini-Course

May 22, 2024

Authors: Roger Melko, Ejaaz Merali and Mohamed Hibat-Allah

Start by reading the text and code below.
You will  find a total of 6 exercises (plus 2 additional "bonus" challenge exercises) in the "Exercises" section below.

## Overview and goal

The aim of this tutorial is to show you how to train a Restricted Boltzman Machine (RBM) to reconstruct the wavefunction of an array of Rydberg atoms whose exact state is unknown. We will show an example with an array of 8 atoms. Each of the 8 atoms can be in the groundstate (0 state) or Rydberg state (1 state). You can read more about Rydberg atoms arrays here: https://arxiv.org/pdf/2002.07413. You can find the expression of the Hamiltonian in Section. IV.

## The Training Data

Now $\Omega$ and $\delta$ are 2 parameters of the Rydberg Hamiltonian (see https://arxiv.org/pdf/2107.00766.pdf).
In this example, we fix $\Omega=1.0$.
Then for each
$\delta \in \{ 1.00, 1.02, 1.04, 1.06, 1.08, 1.10, 1.12, 1.14, 1.16, 1.18, 1.20 \}$,
we obtain 10,000 measurements of the state of the 8 atoms.
Thus there are 11 training datasets for each $\delta$ and each training dataset has 10,000 rows (for the 10,000 measurements) and 8 columns as there are 8 Rydberg atoms in the array.

The training data is stored at https://github.com/PIQuIL/EssiQQurke/tree/main/data.
The training data file for each $\delta$
 is given in the `data/nY=8` directory in the `_samples.csv` files.

## Training the RBM

The RBM is trained using [QuCumber](https://github.com/PIQuIL/QuCumber). To train we need to specify the number of visible units and some hyperparameters including the number of hidden units.
As we have an array of 8 atoms, our input vectors will have 8 elements, and there are 8 visible units.
The number of hidden units and the other hyperparameters can be varied for each $\delta$ to obtain better solutions.
More information about the hyperparameters is given in the relevant section of the code below.

## Checking the accuracy of the model

After training is done, we get weights and biases of our trained RBM.
Recall that we do not know the actual wavefunction.
Therefore we cannot compute fidelity, which is a standard matric to evaluate the performance of a model.
However, from the original measurements one point functions and two point functions have been calculated. (Note: the one point function and two point functions are described in detail in the relevant section of the code below).
Now from the trained RBM (for a particular $\delta$) we generate `num_samples` = 10,000 samples.
From those samples we calculate the one point and two point function data. It can be seen that the data we calculate for the one point and two point function are close to the data in the `_1_pt_fn.csv` and `_2_pt_fn.csv` files for a particular $\delta$.
We use these 2 methods to check how well the distributiton of the samples generated by the RBM resembles the distribution of the measurement.

## Directory structure

The code below implements the wavefunction reconstruction example for an array of 8 atoms.

On https://github.com/PIQuIL/EssiQQurke/:
* The `data/` directory contains the training data for different systems which are labelled as `'nY=[number of atoms in the array]'`. Now the `'data/nY =[num_of_atoms_in_array]'` directory contains 3 sets of files for each $
delta$, which are the one point function, two point function and the samples.

* The `output/` directory contains some of the outputs produced by the code, which are:

  1. `reconstructedSample.txt`: This contains the new sampled measurement data produced by the trained RBM.
  2. `reconstructedStateAmplitude.txt`: This contains the amplitudes of the reconstructed quantum state in the computational basis.
  3. `rydberg_data.pt`: contains the parameters of the trained RBM.

## Remarks

In this tutorial we train an RBM using measurement data and then sample from the trained RBM to generate more data.
The generated data is used to reconstruct the unknown wavefunction.
Lastly note that amplitude computation may take some time and thus it may be wise to not do amplitude calculation for $nY>8$
 systems.

## Exercises


1. We have seen in Lecture 1 that the RBM probability distribution is given by:
$$P_{\rm RBM} (\sigma_1^z,\sigma_2^z, ..., \sigma_{nY}^z, h_1, \ldots, h_M)  = \frac{\exp ( \sum_{i=1}^{nY} a_i \sigma_i^z + \sum_{j = 1}^M b_j h_j + \sum_{j,i = 1}^{nY,M} W_{ij} h_i \sigma^z_j  ) }{ Z_{\boldsymbol{\lambda}}},$$ where $\sigma_i^z \pm 1$ and $h_j \pm 1$. Show that the marginalized RBM probability with the $nY$ visible variables and the $M$ hidden variables is given by: $$P_{\rm RBM} (\sigma_1^z,\sigma_2^z, ..., \sigma_{nY}^z)  = \frac{\exp ( \sum_{i=1}^{nY} a_i \sigma_i^z ) \prod_{i=1}^M \cosh (b_i + \sum_{j = 1}^{nY} W_{ij} \sigma^z_j) }{ Z_{\boldsymbol{\lambda}}},$$
up to a constant that does not depend on the parameters.

2. Show that the KL divergence between the true probability distribution and the RBM probability we saw in class is given by:

$$D_{KL} (P || P_{RBM}) \approx -H_{\text{data}} - \frac{1}{|D|} \sum_{\boldsymbol{\sigma} \in |D|} \log P_{RBM}(\boldsymbol{\sigma}), $$
where $\boldsymbol{\sigma} =  (\sigma_1^z,\sigma_2^z, ..., \sigma_{nY}^z)$ and $H_{\text{data}}$ is the entropy of the data. From this expression, deduce that minimizing the KL divergence is the same as maximizing the likelihood of the data (i.e. the product of all probabilities in the dataset).

3. Using question 1 and 2, show that:

$$D_{KL} (P || P_{RBM}) \approx -H_{\text{data}} - \frac{1}{|D|} \sum_{\boldsymbol{\sigma} \in |D|} ( \sum_{i=1}^{nY} a_i \sigma_i^z  +  \sum_{i=1}^M \log \cosh (b_i + \sum_{j = 1}^{nY} W_{ij} \sigma^z_j) ) + \log(Z_{\boldsymbol{\lambda}}).$$
Deduce the expression of the gradients of $D_{KL}$ and infer which terms is intractable to estimate in practice.

You can learn more here how Contrastrive Divergence (CD) can solve the intractability issue here: https://qucumber.readthedocs.io/en/stable/_static/RBM_tutorial.pdf

4.    Train the RBM for $nY=8$ and $\delta=1.14$. How do various hyperparameters affect the quality of your reconstruction?
5.    Train the RBM on larger system size data (try at least up to 16). Adjust your hyperparameters to ensure high quality reconstruction. Plot the number of hidden units required for good representation as a function of $nY$.
6.   Refer to the QuCumber tutorial on [sampling and calculating observables](https://github.com/PIQuIL/QuCumber/blob/master/examples/Tutorial4_DataGeneration_CalculateObservables/tutorial_sampling_observables.ipynb).
Calculate the off-diagonal observable $\langle \sigma_x \rangle$ (the in-plane magnetization).
In what limits is it possible to confirm your result?

**Additional "bonus" challenge exercises:**

You likely won't have time to complete these exercises during the tutorial but are encouraged to try them afterwards if you are interested.

1.   Calculate the second Renyi entropy $S_2$ as a function of the size of a sub-region A.
Try for one detuning parameter far away from criticality, and one close to criticality.
2.   At the critical point, extract the central charge $c$ of the Conformal Field Theory (CFT) corresponding to the critical detuning.
Extract for different finite-size lattices, and extrapolate.
How does your result compare to the theoretical value?



## Code

In [ ]:
#Run to install qucumber (version 1.3.3):
!pip install qucumber

In [ ]:
# First we import the libraries
# Please make sure you have the required libraries.
# You can use pip or conda to get them.

import numpy as np

from qucumber.nn_states import PositiveWaveFunction
from qucumber.callbacks import MetricEvaluator

import qucumber.utils.training_statistics as ts
import qucumber.utils.data as data
import qucumber

import torch

# set random seed on cpu but not gpu, as gpu is not used
qucumber.set_random_seed(1234, cpu=True, gpu=False)

In [ ]:
# Now we get the training data
# Training data is given in the data/nY=8/ directory for each delta
# in the '...samples.csv' files.
# So the delta needs to be specified below

#-------------------- Specify delta and nY ---------------------#
delta = "1.14"
nY = "8" # nY refers to the number of atoms in the array
# For the nY = 8 system case:
# delta is any of 1.00, 1.02, 1.04, 1.06, 1.08, 1.10, 1.12, 1.14, 1.16, 1.18, 1.20
# For nY > 8:
# delta is any of 1.00, 1.04, 1.08, 1.12, 1.16, 1.20, 1.28
#---------------------------------------------------------------#

train_data_file = "δ="+delta+"_samples.csv"
link_name = "https://raw.githubusercontent.com/PIQuIL/EssiQQurke/main/data/nY="+nY+"/"+train_data_file
!wget "$link_name" #download data to Google Colab

train_data = data.load_data(train_data_file)

# The training data is stored in train_data[0]
# and the dimension of the data is:
train_data[0].shape

In [ ]:
# So now we have the training data. Our goal is to train a
# Restricted Boltzman Machine using this training data. We use
# QuCumber to create an instance of an RBM. First we need to specify the
# number of visible nodes (nv) and number of higgen nodes (nh).
# As we have an array of 8 atoms, we have 8 inputs and so nv = 8
nv = train_data[0].shape[-1]

# Number of hidden nodes of an RBM is a hyperparameter which depends on the
# data we are using and which can be varied to get optimal result.
nh = 2

# Finally we create an RBM with nv visible nodes and nh  hidden nodes.
nn_state = PositiveWaveFunction(num_visible=nv, num_hidden=nh, gpu=False)

In [ ]:
# Below we have more hyperparameters of the RBM model. Like the number of
# hidden nodes, the parameters below can be varied to get optimal results.
# The following hyperparameters seemed to work quite well. Note that when
# you have different data, then different values of the hyperparameter will give
# better solutions.
# Further description of the parameters (if you are curious) is in:
# https://qucumber.readthedocs.io/en/stable/quantum_states.html?highlight=fit#qucumber.nn_states.PositiveWaveFunction.fit

epochs = 500
pbs = 100 # batch size used for the positve phase term
nbs = pbs # batch size used for the negative phase term
lr = 0.001 # For nY > 8 systems lr = 0.0065 can be used as it works quite well.
k = 10

# Now we train our RBM using the above parameters.
# NOTE: the training process will take 1-3 minutes

nn_state.fit(
    train_data[0],
    epochs=epochs,
    pos_batch_size=pbs,
    neg_batch_size=nbs,
    lr=lr,
    k=k,
    time=True,
)

In [ ]:
# After training is complete, we save the parameters of the trained RBM below
# as rydberg_data.pt
nn_state.save("rydberg_data.pt")
torch.load("rydberg_data.pt")
# Below we have the parameters (weights and biases) of the trained RBM:

In [ ]:
# Now we have our trained RBM. Here we reconstruct the wavefunction by sampling from the RBM.
# Let us see step by step how it works -

# The first step is to sample from our trained RBM using QuCumber.
# We then get num_samples samples which is stored in the variable 'samples'
num_samples = 10000
samples = nn_state.sample(num_samples = num_samples , k = 10)

# Now we print out the samples in 'output/reconstructedSample.txt'
sampleList = samples.tolist() # convert to list for convenience
sampleList
with open('reconstructedSample.txt', 'w') as fp:
    for item in sampleList:
        fp.write("%s\n" % item)


In [ ]:
######## OPTIONAL CELL ########

## -------------------------------------------------------------------- ##
## This block of code computes amplitudes. For nY > 8, it is not necessary
## to run this block as computation takes a lot of time.
## -------------------------------------------------------------------- ##

# From the samples we obtain the amplitudes of each of the basis states.
# Note that here we have an 8 atom array. Hence, there are 2^8 basis states
# which are: |00000000>, |00000001>, ..., |11111111>

# Now say X is any of the states from {|00000000>, |00000001>, ..., |11111111>}
# Now, the amplitude of a state X is computed by taking the
# square root of the number of states X present in
# our produced sample in 'output/reconstructedSample.txt'.

# Therefore the reconstructed sample is like:
#    |psi> = amplitudeList[0] |00000000> + amplitudeList[1] |00000001> + ... + amplitudeList[255] |11111111>
# where amplitudeList is defined below.

# The following functions below finds the amplitudes in the way described above, stores the
# amplitudes in amplitudeList and then also
# prints out the amplitudes in 'reconstructedSample.txt'

def amplitude(sample):
    return np.sqrt(sampleList.count(sample)/num_samples)

def getBinaryString(i):
    sites = nv # nv = 8
    getbinary = lambda x, n: format(x, 'b').zfill(n)
    tempStr = getbinary(i, sites)
    return tempStr

# Python code to convert string to list character-wise
def ConvertToList(string):
    list1=[]
    list1[:0]=string
    return list1

amplitudeList = []
def getAmplitude(sample):
    for i in range (2**nv):
        binaryString = getBinaryString(i)
        strList = ConvertToList(binaryString)
        tempStr = str(amplitude(list(map(int, strList))))
        amplitudeList.append(tempStr)
    return 0

getAmplitude(sampleList)
with open('reconstructedStateAmplitudes.txt', 'w') as fp:
    for item in amplitudeList:
        fp.write("%s\n" % item)

In [ ]:
# So now we have obtained the reconstructed wavefunction.
# Now in the 'data' directory for each delta we have 1_pt_fun and 2_pt_fn
# 1_pt_fn is the one point function and
# 2_pt_fn is the 2 point function.

# 1_pt_fn is the average occupation of each site. Note that
# there are 8 sites as there are 8 atoms. Occupation of each site
# refers to the proportion of atoms in the up state (state 1)
# in each state.

# Now we get average occupation of each site by finding the
# number of atoms in the 1 state in each site and then
# by dividing by the number of the produced samples
occupation = [0]*nv
for i in range(len(sampleList)):
    j = 0
    for j in range(nv):
        occupation[j] = sampleList[i][j] + occupation[j]
for i in range(nv):
    occupation[i] = occupation[i]/len(sampleList)

# Thus the average occupation per site of each site for the reconstructed state is given below
# This data can be compared to data/nY=8/δ=delta_1_pt_fn.csv for the relevant delta

# (Note: The list in data/nY=8/δ=delta_1_pt_fn.csv was obtained from
# the sample in data/nY=8/δ=delta_samples.csv)

# Thus the data in 'occupation' and 'data/nY=8/δ=delta_1_pt_fn.csv' are expected to be close (which they are)
occupation

In [ ]:
# Now we look at the 2 point function
# 2 point function is the covariance matrix for occupations.
# It is found by 2_pt_fn = outerProdT1 - OuterProdT2

# outerProdT1 is the average of the outer product of the spin vectors with themselves
# So we first find the sum of the outerproducts of the spin vectors with themselves.
# Then we take the average
outerProdT1 = np.zeros((nv,nv))
for i in range(len(sampleList)):
    outerProdT1 = outerProdT1 + np.outer(sampleList[i], sampleList[i])
outerProdT1 = outerProdT1/len(sampleList)

# outerProd2 is simply the outer product of the occupation vector
# with itself. Note that we already found the occupation vector for the
# one point function

outerProdT2 = np.outer(occupation,occupation)
outerProdT2

#Hence we have:
TwoPointFunc = outerProdT1 - outerProdT2

# The TwoPointFunction data for the reconstructed state is given below.
# This matrix can be compared to data/nY=8/δ=delta_2_pt_fn.csv for the relevant delta

# (Note: The matrix in data/nY=8/δ=delta_2_pt_fn.csv was obtained from
# the sample in data/nY=8/δ=delta_samples.csv)

# Thus the matrix in 'TwoFuncPoint' and 'data/nY=8/δ=delta_2_pt_fn.csv' are expected
# to be nearly equal (which they are)
TwoPointFunc

## Question 5

In [ ]:
#Answer question 5 here

## Question 6

In [ ]:
#Answer question 6 here